In [1]:
import pandas as pd
df = pd.read_csv('data/ZILLOWforMom.csv')

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df = df.drop(columns=['Unnamed: 0'])
df.head()

,property_id,listing_id,prop_type,list_date,last_update,year_built,beds,baths_full,garage,baths_half,prop_status,price,baths,rank,is_new_construction,photo_count,price_reduced_date,line,unit,city,state_code,postal_code,county,neighborhood_name,ListingAgent,RealEstateCompany,email,PrimaryNumber,Property Size(Sq. Ft)
0,O6713667032,2913863195,condo,2020-03-05T17:29:12Z,2020-07-27T17:04:27Z,2016,2,2,1,0,not_for_sale,675000,2,7,False,37,2020-06-12T13:17:46.535Z,525 Water St SW Unit 228,Unit 228,Washington,DC,20024,District of Columbia,"Southwest Washington - Washington,DC",Alex Venditti,Coldwell Banker Residential Brokerage - Dupont...,900613.lead@leads.leadrouter.com,703-824-4800,860
1,O5070215644,2917917967,condo,2020-07-02T04:16:46Z,2020-07-29T17:59:05Z,1963,2,1,0,1,not_for_sale,387000,2,10,False,19,0,1301 N Van Dorn St Unit 1301,Unit 1301,Alexandria,VA,22304,Alexandria City,"Seminary Hill - Alexandria,VA",Ali Velasco,Keller Williams Capital Properties,0,877-705-5080,2847
2,O6542558516,2917660384,condo,2020-06-25T04:16:01Z,2020-07-29T10:37:24Z,1929,2,1,0,0,not_for_sale,511300,1,20,False,22,0,223 18th St SE Apt 4,Apt 4,Washington,DC,20003,District of Columbia,"Hill East - Washington,DC",Amanda Briggs,COMPASS,listings@compass.com,202-333-3320,581
3,O5955316599,2916171825,single_family,2020-05-13T04:07:18Z,2020-07-28T15:57:28Z,1956,4,2,0,0,not_for_sale,535000,2,27,False,27,0,6317 Colette Dr,0,Alexandria,VA,22315,Fairfax,0,Ana Yassawy,"Big Apple Realty, Llc",0,703-671-1000,2100
4,O6279621827,2917346192,condo,2020-06-16T12:08:41Z,2020-07-27T19:45:27Z,1927,2,2,0,1,not_for_sale,595000,3,37,False,44,0,329 Rhode Island Ave NE Apt 301,Apt 301,Washington,DC,20002,District of Columbia,"Northeast Washington - Washington,DC",Andrew Riguzzi,COMPASS,marketingdc@compass.com,301-249-1600,888


In [3]:
#Dropping non-string columns
df = df.drop(columns=['county', 'neighborhood_name', 'ListingAgent', 'RealEstateCompany', 'email', 'PrimaryNumber' , 
                      'is_new_construction', 'prop_status', 'list_date', 'last_update', 'property_id', 'unit', 'line', 
                      'postal_code', 'price_reduced_date','listing_id', 'city'])
df.head()

,prop_type,year_built,beds,baths_full,garage,baths_half,price,baths,rank,photo_count,state_code,Property Size(Sq. Ft)
0,condo,2016,2,2,1,0,675000,2,7,37,DC,860
1,condo,1963,2,1,0,1,387000,2,10,19,VA,2847
2,condo,1929,2,1,0,0,511300,1,20,22,DC,581
3,single_family,1956,4,2,0,0,535000,2,27,27,VA,2100
4,condo,1927,2,2,0,1,595000,3,37,44,DC,888


In [4]:
import pandas as pd
import sys
import sqlite3
import numpy as np
!pip install sklearn --upgrade
!pip install joblib

Requirement already up-to-date: sklearn in c:\users\bocli\anaconda3\lib\site-packages (0.0)


In [5]:
#Checking all possible values of prop_type for changing the data
df['prop_type'].value_counts()

condo            123
single_family     75
land               1
Name: prop_type, dtype: int64

In [7]:
#Checking all possible values of state_code for changing the data
df['state_code'].value_counts()

VA    96
DC    85
MD    18
Name: state_code, dtype: int64

In [8]:
#changing city and prop_type and state_code to ordinal values
cleanup_proptype = {"prop_type": {'condo':1, 'single_family':2, 'land': 3}}
cleanup_statecode = {'state_code':{'DC':1, 'VA':2, 'MD':3}}

In [9]:
df.replace(cleanup_proptype, inplace=True)

In [10]:
df.head()

,prop_type,year_built,beds,baths_full,garage,baths_half,price,baths,rank,photo_count,state_code,Property Size(Sq. Ft)
0,1,2016,2,2,1,0,675000,2,7,37,DC,860
1,1,1963,2,1,0,1,387000,2,10,19,VA,2847
2,1,1929,2,1,0,0,511300,1,20,22,DC,581
3,2,1956,4,2,0,0,535000,2,27,27,VA,2100
4,1,1927,2,2,0,1,595000,3,37,44,DC,888


In [11]:
df.replace(cleanup_statecode, inplace=True)

In [12]:
#Dataframe ready for machine learning analysis
df.head()

,prop_type,year_built,beds,baths_full,garage,baths_half,price,baths,rank,photo_count,state_code,Property Size(Sq. Ft)
0,1,2016,2,2,1,0,675000,2,7,37,1,860
1,1,1963,2,1,0,1,387000,2,10,19,2,2847
2,1,1929,2,1,0,0,511300,1,20,22,1,581
3,2,1956,4,2,0,0,535000,2,27,27,2,2100
4,1,1927,2,2,0,1,595000,3,37,44,1,888


In [23]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import MinMaxScaler,StandardScaler, Normalizer
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import classification_report as report
from random import randint
from sys import maxsize
import warnings
from sklearn.exceptions import DataConversionWarning,UndefinedMetricWarning
warnings.filterwarnings(action='ignore',category=DataConversionWarning)
warnings.filterwarnings(action='ignore',category=UndefinedMetricWarning)

#pipeline: 1) scales model 2) reduces dimensionality 3) tests the model

#this function takes an arugent n_features representing the total number of features to keep
#by iterating through n_features=1 to n_features = max_features we will determine the best set of features
"""
    SUMMARY:  Creates a pipeline with the given data preprocessors and model and then tests the model using the pipeline
    
    
    
    IN: n_features = denotes the number of features, n_features, to select from the overall data set
                        there is a total of 40 columns in the input data and we may not want all of that data
                        some of the featuress could be 'noise' and not be useful to predicting the output.
                        We will use Principal Component Analysis, by way of the SelectKBest function and the f_classif function
                        do decide which selection of features is optimal
        
        testmodel = denotes the model we wish to test in this test run.  could be any classification model that supports
                        the functions "fit" and "fit_transform"
                        

            
    OUT:  Returns the overall model precision

"""

def RunTest(n_features,testmodel):
    X= df.drop(columns=["prop_type"],inplace=False)
    y= df["prop_type"].values.reshape(-1,1) # same as  y = [ df["koi_disposition"].values ]
    X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=randint(0,2**32 - 1))
    #testscaler=MinMaxScaler(feature_range=(-1,1),copy=True)
    testscaler=StandardScaler()
    normalizer=Normalizer()
    testpca=SelectKBest(f_classif,k="all")
    mypipeline=Pipeline([('featurescale',testscaler),("mynormalizer",normalizer), ("anova",testpca), ("model",testmodel)])    
    mypipeline.fit(X_train,y_train)
    predictions=mypipeline.predict(X_test)
    return report(y_test,predictions,output_dict=True)
    



precisions={}
models={KNN(n_neighbors=1):"knn",RandomForestClassifier():"rff",SVC():"svc", LogisticRegression():"lin",
       } #chooses all models from dict list beforehand
for model,name in models.items():
    for n_features in range(3,1,-1): #loops through each model and every step of the way it throws out the worst column
        score=RunTest(n_features,model)["weighted avg"]['precision']
        n=n_features
        precisions[float(score)]=(name,n)
answer=max(precisions)
print(answer,str(precisions.get(answer)))

0.8729166666666667 ('lin', 2)


C:\Users\bocli\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\bocli\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\bocli\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\bocli\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify 

In [24]:
df.head()

,prop_type,year_built,beds,baths_full,garage,baths_half,price,baths,rank,photo_count,state_code,Property Size(Sq. Ft)
0,1,2016,2,2,1,0,675000,2,7,37,1,860
1,1,1963,2,1,0,1,387000,2,10,19,2,2847
2,1,1929,2,1,0,0,511300,1,20,22,1,581
3,2,1956,4,2,0,0,535000,2,27,27,2,2100
4,1,1927,2,2,0,1,595000,3,37,44,1,888
